In [1]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic

In [2]:
import numpy as np

a = np.array([32.49, -39.96,-3.86])
b = np.array([31.39, -39.28, -4.66])
c = np.array([31.14, -38.09,-4.49])

def calc_angle(a, b, c):

    a = np.array([a.x, a.y, a.z])
    b = np.array([b.x, b.y, b.z])
    c = np.array([c.x, c.y, c.z])


    ba = a - b
    bc = c - b

    print("Ba:" + str(ba))
    print("BC:" + str(bc))

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

def get_dist(a, b):
    return np.linalg.norm(np.array([b.x, b.y, b.z]) - np.array([a.x, a.y, a.z]))

In [4]:
import math
import traceback

cap = cv2.VideoCapture(0)

# Set up OpenCV window
# Changed the window title to 'Deadlift Form Corrector'
cv2.namedWindow('Deadlift Form Corrector', cv2.WINDOW_NORMAL)
# Resized the window to 1200x600 pixels for a wider view
cv2.resizeWindow('Deadlift Form Corrector', 1200, 600)

# Curl counter variables
counter = 0
stage = None

alr_ran = False
og_l_should_pos = 0
og_r_should_pos = 0

state = "none"
reps = 0



# Setup mediapipe instance
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic, mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
        results_face = holistic.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        stage = []
        # Extract landmarks
        try:
            #print("Testing")
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            
            #print(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])
            hip_dist = get_dist(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
            ankle_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value], landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

            wrist_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value], landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
            shoulder_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
            
            l_shoulder_hip_dist = get_dist(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
            r_shoulder_hip_dist = get_dist(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value])


            right_elbow_angle = calc_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value], 
                                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
            left_elbow_angle = calc_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value], landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value], 
                                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

            right_knee_angle = calc_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value], landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value], 
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

            left_knee_angle = calc_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value], 
                                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])


            new_state = ""
            if left_knee_angle >= 120 or right_knee_angle >= 120:
                new_state = "up"

            if left_knee_angle <= 80 or right_knee_angle <= 80:
                new_state = "down"
            
            if state == "up" and new_state == "down":
                reps += 1
            
            state = new_state


            #print("Right knee:" + str(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]))
            #print("Right hip:" + str(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]))

            # cv2.putText(image, "Hip dis:" + str(hip_dis), 
            #                 (200,200), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            # cv2.putText(image, "Left Elbow Angle:" + str(round(left_elbow_angle, 4)), 
            #                 (300,300), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # cv2.putText(image, "Left Knee Angle:" + str(round(left_knee_angle, 4)), 
            #                 (400,400), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            

            #These are the arm angles to see if the user is spreading/moving their arms too much
            # cv2.putText(image, str(l_arm_angle), 
            #                 tuple(np.multiply(l_shoulder, [640, 480]).astype(int)), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            # cv2.putText(image, str(r_arm_angle), 
            #                 tuple(np.multiply(r_shoulder, [640, 480]).astype(int)), 
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                      )

            
            # Checking Logic
            
            if hip_dist / ankle_dist > 0.65:
                stage.append("Your legs are too narrow")
            if hip_dist / ankle_dist < 0.4:
                stage.append("Your legs are too wide") 
            if shoulder_dist / wrist_dist > 1:
                stage.append("Move your arms farther apart on the bar")
            if shoulder_dist / wrist_dist < 0.58:
                stage.append("Move your arms close together on the bar")  

            if left_elbow_angle < 120:
                stage.append("Keep your arms straight, it can strain your elbows")
            
            if left_knee_angle < 50 or right_knee_angle < 50:
                stage.append("Don't bend down too much, it can strain your knees")
            
            if len(stage) == 0:
                    stage.append("Great Form!")# Render video frame to the left


                       
        except Exception:
            traceback.print_exc()
        
        # Render curl counter

# Setup s# Render video frame to the left
        # Changed the color of the status box to blue
        #cv2.rectangle(image, (0, 0), (720, 130), (0, 0, 255), -1)

        # Render project title
        cv2.putText(image, 'MotionMentor Deadlift Form Corrector', (540, 40),  # Centered at the top
                    cv2.FONT_HERSHEY_DUPLEX, 1.3, (255, 255, 255), 2, cv2.LINE_AA)

        # Render blue box with text at the top
        #cv2.rectangle(image, (0, 0), (1200, 60), (70, 130, 180), -1)  # Blue box
        # Stage data
        # cv2.putText(image, 'MOTION MENTOR', (550,30), 
        #             cv2.FONT_HERSHEY_TRIPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.putText(image, "Reps: " + str(reps), 
                            (1000,100), 
                            cv2.FONT_HERSHEY_DUPLEX, 1.8, (255,255,255), 2, cv2.LINE_AA)

        if(len(stage) == 0):
            continue
        elif stage[0] == "Great Form!":
            cv2.putText(image, stage[0], 
                            (30,100), 
                            cv2.FONT_HERSHEY_DUPLEX, 1.8, (0,255,0), 2, cv2.LINE_AA)
        else:
            i = 0
            for x in stage:
                cv2.putText(image, x,
                            (30,120 + i*60), 
                            cv2.FONT_HERSHEY_DUPLEX, 1.5, (0,0,255), 2, cv2.LINE_AA)
                i += 1

        print(stage)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results_face.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2) 
                                 )               
                                 
        
        cv2.imshow('Deadlift Form Corrector', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
        

I0000 00:00:1706983682.123791       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1
I0000 00:00:1706983682.140211       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose

Ba:[0.02897319 0.02065948 0.06498086]
BC:[ 0.01100719  0.04035544 -0.06839483]
Ba:[0.03021707 0.02110749 0.08302386]
BC:[ 0.03195758  0.0040431  -0.06891382]
Ba:[0.02763486 0.02440743 0.12541688]
BC:[-0.05775657 -0.00437792  0.02561288]
Ba:[ 0.0302416  -0.00281452  0.14904305]
BC:[-0.05035175 -0.06374095  0.05070612]
['Your legs are too narrow', 'Move your arms farther apart on the bar']
Ba:[ 0.00703011 -0.01489113  0.03529846]
BC:[ 0.02301018  0.04205972 -0.00227195]
Ba:[ 0.00761992 -0.00699863  0.05993201]
BC:[ 0.03352854 -0.00395855 -0.02624322]
Ba:[0.03728974 0.03431656 0.10097974]
BC:[-0.05729908  0.00569691  0.04924305]
Ba:[ 0.03738802 -0.00134344  0.11157579]
BC:[-0.04779494 -0.0630751   0.05288078]
['Your legs are too narrow', 'Move your arms farther apart on the bar', 'Keep your arms straight, it can strain your elbows']
Ba:[0.03314263 0.02908809 0.01585848]
BC:[-0.04795282  0.00024718  0.01471043]
Ba:[0.04059759 0.0175367  0.00449066]
BC:[-0.03400873 -0.01133187 -0.00947439]


Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'


Ba:[ 0.00636405 -0.00741899  0.01661271]
BC:[ 0.00212815  0.00906914 -0.00481398]
Ba:[ 0.01225686 -0.00769776  0.0108041 ]
BC:[ 3.80706787e-03 -8.40425491e-06 -2.09640632e-02]
Ba:[ 0.01477185 -0.0121516   0.01195153]
BC:[-0.02078947 -0.00893557  0.01089328]
Ba:[ 0.01965559 -0.00834614  0.00895929]
BC:[-0.01737797 -0.01277965  0.00745436]
['Your legs are too narrow', 'Move your arms farther apart on the bar', 'Keep your arms straight, it can strain your elbows']


Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'


...

Ba:[-0.04569227 -0.04471949 -0.11911335]
BC:[-0.00135827 -0.01501571  0.08116361]
Ba:[-0.04553452 -0.00438988 -0.01923646]
BC:[ 0.02075076  0.01016739 -0.04056983]
Ba:[ 0.07795486 -0.15716085  0.13319502]
BC:[-0.03163889  0.22792286 -0.07647637]
Ba:[-0.03109002 -0.21829943  0.15317693]
BC:[ 0.03155631  0.22176409 -0.08180721]
['Move your arms close together on the bar', 'Keep your arms straight, it can strain your elbows']


Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ip

Ba:[-0.00152916 -0.04073352 -0.02157735]
BC:[ 0.01686168 -0.02051519 -0.00546943]
Ba:[ 0.01889074 -0.08057301 -0.00127906]
BC:[-0.02069044  0.05002399 -0.03801222]
Ba:[ 0.01350012 -0.08562079  0.05299914]
BC:[-0.01949796  0.13296281  0.06138046]
Ba:[ 0.02236214 -0.08924195  0.043477  ]
BC:[-0.02497831  0.10180409  0.03826863]
['Your legs are too narrow', 'Move your arms close together on the bar']


Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ipykernel_80503/234833088.py", line 45, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'
Traceback (most recent call last):
  File "/var/folders/j4/q08w9xt532d93xjh2hr_w9b00000gn/T/ip

KeyboardInterrupt: 